# Resistive Wall Impedance

This notebook demonstrates the numerical calculation of longitudinal resistive wall impedance $Z(k)$ and wakefield $W(z)$ for flat (parallel plate) and round (circular pipe) geometries.

**References:**
- K. Bane and G. Stupakov, "Resistive wall wakefield in the LCLS undulator beam pipe," SLAC-PUB-10707 (2004)
- G. Stupakov et al., Phys. Rev. ST Accel. Beams 18, 034402 (2015)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from pmd_beamphysics.wakefields import ResistiveWallImpedance, ResistiveWallWakefield

## Flat Geometry

Create a flat (parallel plate) geometry impedance model by specifying the half-gap and material properties:

In [ ]:
imp_flat = ResistiveWallImpedance(
    radius=4.5e-3,  # half-gap for flat geometry [m]
    conductivity=2.4e7,  # S/m
    relaxation_time=8e-15,  # s
    geometry="flat",
)
imp_flat

In [ ]:
imp_flat.plot_impedance()

In [ ]:
imp_flat.plot_wakefield()

## Round Geometry

Create a round (circular pipe) geometry impedance model:

In [ ]:
imp_round = ResistiveWallImpedance(
    radius=4.5e-3,  # pipe radius [m]
    conductivity=2.4e7,  # S/m
    relaxation_time=8e-15,  # s
    geometry="round",
)
imp_round

In [ ]:
imp_round.plot_impedance()

In [ ]:
imp_round.plot_wakefield()

---

## Comparing Flat and Round Geometries

For detailed comparisons, we can compute impedance and wakefield arrays directly and create custom plots.

In [ ]:
# Compute impedance arrays
ks = np.linspace(0, 3e5, 100)
Zk_flat = imp_flat.impedance(ks)
Zk_round = imp_round.impedance(ks)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

ax = axes[0]
ax.plot(ks * 1e-3, np.real(Zk_flat), label="Flat")
ax.plot(ks * 1e-3, np.real(Zk_round), "--", label="Round")
ax.set_xlabel(r"$k$ (1/mm)")
ax.set_ylabel(r"Re[$Z(k)$] (Ω/m)")
ax.legend()

ax = axes[1]
ax.plot(ks * 1e-3, np.imag(Zk_flat), label="Flat")
ax.plot(ks * 1e-3, np.imag(Zk_round), "--", label="Round")
ax.set_xlabel(r"$k$ (1/mm)")
ax.set_ylabel(r"Im[$Z(k)$] (Ω/m)")
ax.legend()

plt.suptitle("Impedance Comparison: Flat vs Round Geometry")
plt.tight_layout()

In [ ]:
# Compute wakefield arrays (slow due to numerical integration)
zs = np.linspace(0, 200e-6, 30)
Wz_flat = imp_flat.wakefield(zs, k_max=1e6)
Wz_round = imp_round.wakefield(zs, k_max=1e6)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(zs * 1e6, Wz_flat * 1e-12, label="Flat")
ax.plot(zs * 1e6, Wz_round * 1e-12, "--", label="Round")
ax.set_xlabel(r"$z$ (µm)")
ax.set_ylabel(r"$W(z)$ (V/pC/m)")
ax.legend()
ax.set_title("Wakefield Comparison: Flat vs Round Geometry")

---

# Comparison with Pseudomode Model

The numerical integration approach above is accurate but slow. For production use, the `ResistiveWallWakefield` class provides a much faster **pseudomode** approximation based on analytical fits from SLAC-PUB-10707.

The pseudomode models the wakefield as a damped sinusoid:

$$W(z) = A \, e^{d z} \sin(k z + \phi)$$

where the parameters $(A, d, k, \phi)$ are derived from polynomial fits to numerical results.

In [ ]:
# Create pseudomode wakefield models with same parameters
wake_flat = ResistiveWallWakefield(
    radius=4.5e-3,  # half-gap for flat geometry
    conductivity=2.4e7,
    relaxation_time=8e-15,
    geometry="flat",
)

wake_round = ResistiveWallWakefield(
    radius=4.5e-3, conductivity=2.4e7, relaxation_time=8e-15, geometry="round"
)

print("Flat geometry pseudomode:")
print(wake_flat.pseudomode)
print("\nRound geometry pseudomode:")
print(wake_round.pseudomode)

In [ ]:
# Evaluate pseudomode wakefields (very fast!)
zs_fine = np.linspace(0, 200e-6, 200)

# Note: pseudomode expects negative z (trailing the source)
Wz_pseudo_flat = wake_flat.pseudomode(-zs_fine)
Wz_pseudo_round = wake_round.pseudomode(-zs_fine)

In [ ]:
# Compare numerical integration with pseudomode fit
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

ax = axes[0]
ax.plot(zs * 1e6, Wz_flat * 1e-12, "o", label="Numerical", markersize=5)
ax.plot(zs_fine * 1e6, Wz_pseudo_flat * 1e-12, "-", label="Pseudomode", alpha=0.8)
ax.set_xlabel(r"$z$ (µm)")
ax.set_ylabel(r"$W(z)$ (V/pC/m)")
ax.legend()
ax.set_title("Flat Geometry")

ax = axes[1]
ax.plot(zs * 1e6, Wz_round * 1e-12, "o", label="Numerical", markersize=5)
ax.plot(zs_fine * 1e6, Wz_pseudo_round * 1e-12, "-", label="Pseudomode", alpha=0.8)
ax.set_xlabel(r"$z$ (µm)")
ax.set_ylabel(r"$W(z)$ (V/pC/m)")
ax.legend()
ax.set_title("Round Geometry")

plt.suptitle("Numerical Integration vs Pseudomode Approximation", y=1.02)
plt.tight_layout()

## Performance Comparison

The pseudomode approach is **orders of magnitude faster** than numerical integration.

In [ ]:
%%timeit -n 1 -r 3
# Numerical integration for 20 points
z_test = np.linspace(0, 100e-6, 20)
_ = imp_flat.wakefield(z_test, k_max=1e6)

In [ ]:
%%timeit -n 1000 -r 3
# Pseudomode for 1000 points
z_test = np.linspace(0, 100e-6, 1000)
_ = wake_flat.pseudomode(-z_test)

## When to Use Each Approach

| Method | Speed | Accuracy | Use Case |
|--------|-------|----------|----------|
| `FlatResistiveWallImpedance` / `RoundResistiveWallImpedance` | Slow | High | Validation, research, detailed analysis |
| `ResistiveWallWakefield` (pseudomode) | Very fast | Good | Production simulations, particle tracking |

The pseudomode approximation is excellent for short-range wakefields and is suitable for most beam dynamics applications. Use the numerical integration when:
- Validating the pseudomode fits
- Working with unusual parameter regimes
- Need access to the impedance spectrum $Z(k)$